In [ ]:
!pip install -q torchvision transformers diffusers accelerate controlnet_aux xformers pandas pillow opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your dataset folder (adjust path)
%cd /content/drive/MyDrive/All_Monuments/All_Monuments

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/All_Monuments/All_Monuments


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image

# Paths configuration
images_root = "monuments_2"
depths_root = "monuments_2_depths"  # Directory containing depth maps
samples = []

for root, _, files in tqdm(os.walk(images_root), desc="Processing images"):
    if "monuments_captions.csv" in files:
        captions_df = pd.read_csv(
            os.path.join(root, "monuments_captions.csv"),
            header=None,
            names=["filename", "raw_caption"]
        )

        for _, row in captions_df.iterrows():
            image_path = os.path.join(root, row["filename"])

            if not os.path.exists(image_path):
                continue

            # Get corresponding depth path
            rel_path = os.path.relpath(image_path, images_root)
            depth_path = os.path.join(
                depths_root,
                os.path.splitext(rel_path)[0] + "_depth.png"  # Adjust extension if needed
            )

            samples.append({
                "image_path": image_path,
                "depth_path": depth_path,
                "raw_caption": row["raw_caption"],
            })

# Create DataFrame
df = pd.DataFrame(samples)
print(f"\nTotal samples: {len(df)}")
# Check for any rows where image or depth paths are invalid or missing
invalid_rows = df[df['image_path'].isnull() | df['depth_path'].isnull()]
if not invalid_rows.empty:
    print("Invalid entries found:")
    print(invalid_rows)


df.head()

Processing images: 5it [00:07,  1.53s/it]


Total samples: 1505


,image_path,depth_path,raw_caption
0,monuments_2/Baroque architecture/5d66482a8551f...,monuments_2_depths/Baroque architecture/5d6648...,"Cathedral | Baroque | Stone, wood | Rough, car..."
1,monuments_2/Baroque architecture/iglesia_san_p...,monuments_2_depths/Baroque architecture/iglesi...,Church | Romanesque | Stone | Rough | 12th cen...
2,monuments_2/Baroque architecture/5bb7b650342fc...,monuments_2_depths/Baroque architecture/5bb7b6...,Monument Type: Baroque | Architecture Type: Ch...
3,monuments_2/Baroque architecture/5bc75fefdc08e...,monuments_2_depths/Baroque architecture/5bc75f...,Palace | Neoclassical | Stone | Ornate carving...
4,monuments_2/Baroque architecture/compas_conven...,monuments_2_depths/Baroque architecture/compas...,Monument Type: Statue | Architecture Type: Cla...


In [ ]:
# Manually test loading one image and depth map from the DataFrame
test_index = 0  # Choose an index to test
row = df.iloc[test_index]

# Load image and depth map manually
try:
    image = Image.open(row["image_path"]).convert("RGB")
    print(f"Successfully loaded image: {row['image_path']}")
except Exception as e:
    print(f"Error loading image: {e}")

try:
    depth_map = Image.open(row["depth_path"]).convert("L")
    print(f"Successfully loaded depth map: {row['depth_path']}")
except Exception as e:
    print(f"Error loading depth map: {e}")


Successfully loaded image: monuments_2/Baroque architecture/5d66482a8551f.jpg
Successfully loaded depth map: monuments_2_depths/Baroque architecture/5d66482a8551f_depth.png


In [ ]:
!pip install noise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp311-cp311-linux_x86_64.whl size=56281 sha256=affa665e5a166e47b76dee8852701d87c905a8c1ff5e0efe920d19a3b00910e0
  Stored in directory: /root/.cache/pip/wheels/39/25/2e/af6d1bcc91a8f99af0f651f8718b9ab999720a21c6d4149091
Successfully built noise


In [ ]:
# Modified and Enhanced Training Script for Monument Inpainting
from diffusers import DiffusionPipeline
import matplotlib.pyplot as plt
import argparse
import math
import os
import random
from pathlib import Path
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from huggingface_hub import create_repo, upload_folder
from PIL import Image, ImageDraw, ImageFilter
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer as T5Tokenizer
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionInpaintPipeline, UNet2DConditionModel
from diffusers.loaders import AttnProcsLayers
from diffusers.models.attention_processor import LoRAAttnProcessor
from diffusers.optimization import get_scheduler

In [ ]:
!pip install diffusers transformers

In [ ]:
from diffusers import ControlNetModel

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
import random
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import ImageDraw, ImageFilter
from diffusers import ControlNetModel,StableDiffusionControlNetPipeline

# Define the logger and caption rewriter model
logger = get_logger(__name__)

# Load FLAN-T5 caption rewriter
caption_model_name = "google/flan-t5-small"
caption_tokenizer = T5Tokenizer.from_pretrained(caption_model_name)
caption_model = AutoModelForSeq2SeqLM.from_pretrained(caption_model_name)

def improve_caption(tags):
    """Turn tag list into a natural caption."""
    tags_cleaned = tags.replace("|", ",").replace("  ", " ").strip()
    prompt = f"Create a detailed and natural caption from these tags: {tags_cleaned}"
    inputs = caption_tokenizer(prompt, return_tensors="pt")
    outputs = caption_model.generate(**inputs, max_length=128)
    return caption_tokenizer.decode(outputs[0], skip_special_tokens=True)

def random_damage_mask(im_shape, ratio=0.1, mask_full_image=False):
    """Generate a natural, irregular mask simulating realistic monument damage."""
    mask = Image.new("L", im_shape, 0)
    draw = ImageDraw.Draw(mask)

    if mask_full_image:
        draw.rectangle((0, 0, im_shape[0], im_shape[1]), fill=255)
        return mask

    width, height = im_shape

    # Draw many small irregular spots
    for _ in range(random.randint(10, 20)):
        x = random.randint(0, width)
        y = random.randint(0, height)
        max_radius = int(min(width, height) * ratio * random.uniform(0.3, 0.7))

        for _ in range(random.randint(3, 7)):
            offset_x = random.randint(-max_radius, max_radius)
            offset_y = random.randint(-max_radius, max_radius)
            r = random.randint(3, max_radius)
            bbox = (x + offset_x - r, y + offset_y - r, x + offset_x + r, y + offset_y + r)
            draw.ellipse(bbox, fill=255)

    # Add random "strokes" like cracks
    for _ in range(random.randint(5, 10)):
        start = (random.randint(0, width), random.randint(0, height))
        end = (start[0] + random.randint(-50, 50), start[1] + random.randint(-50, 50))
        thickness = random.randint(1, 5)
        draw.line([start, end], fill=255, width=thickness)

    # Slight blur to make edges soft
    mask = mask.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.5, 2.0)))

    return mask


def prepare_mask_and_masked_image(image, mask, depth_map=None):
    """Enhanced version supporting depth conditioning with ControlNet."""
    # Convert image
    image = np.array(image.convert("RGB"))
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image).to(dtype=torch.float32) / 127.5 - 1.0

    # Convert mask
    mask = np.array(mask.convert("L")).astype(np.float32) / 255.0
    mask = mask[None, None]
    mask = torch.from_numpy(mask)
    mask = (mask >= 0.5).float()

    # Prepare masked image
    masked_image = image * (mask < 0.5)

    # Depth map conditioning (ControlNet)
    if depth_map is not None:
        depth_map = np.array(depth_map.convert("L")).astype(np.float32) / 255.0
        depth_map = depth_map[None, None]
        depth_map = torch.from_numpy(depth_map)

        # Mask the depth map using the same damage mask
        masked_depth = depth_map * (1.0 - mask)  # Masked depth map

        return mask, masked_image, masked_depth

    return mask, masked_image


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel

In [ ]:
import shutil

# Path to your fine-tuned model folder
model_path = "/content/controlnet_pipeline"

# Create a zip of the folder
shutil.make_archive("/content/controlnet_pipeline", 'zip', model_path)


'/content/controlnet_pipeline.zip'

In [ ]:
import os

# Specify the path to your saved pipeline
pipeline_path = "/content/drive/MyDrive/controlnet_pipeline.zip"

# Get the total size of the directory
def get_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            total += os.path.getsize(file_path)
    return total

size_in_bytes = get_size(pipeline_path)
print(f"Pipeline size: {size_in_bytes / (1024**2):.2f} MB")  # Size in MB


Pipeline size: 0.00 MB


In [ ]:
controlnet_model = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16)  # ControlNet for depth conditioning
controlnet_model.save_pretrained("/content/controlnet_pipeline")

In [ ]:
def test_single_sample(dataframe, sample_index=0):
    # Load the ControlNet model for depth conditioning
    controlnet_model = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16)  # ControlNet for depth conditioning

    # Load the Stable Diffusion inpainting pipeline and pass ControlNet as a conditioning model
    pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16)
    pipe.to("cuda")  # Send model to GPU
    # Specify where you want to save the pipeline
    save_path = "/content/fine_tuned_pipeline"
    save_path
      # Save the pipeline to disk
    pipe.save_pretrained(save_path)

    # Get sample from dataframe
    sample = dataframe.iloc[sample_index]

    # Load the image
    image = Image.open(sample["image_path"]).convert("RGB").resize((512, 512))

    # Create random damage mask (simulating monument damage)
    mask = random_damage_mask(image.size, ratio=0.2)

    # Improve caption
    improved_caption = improve_caption(sample["raw_caption"])

    # Load the depth map (assuming depth path is available)
    depth_map = Image.open(sample["depth_path"]).convert("L").resize((512, 512))

    # Masked depth map: Apply the damage mask to the depth map
    depth_array = np.array(depth_map).astype(np.float32) / 255.0
    damage_array = np.array(mask).astype(np.float32) / 255.0
    masked_depth_array = depth_array * (1.0 - damage_array)
    masked_depth = Image.fromarray((masked_depth_array * 255).astype(np.uint8))

    # Prepare the inputs for ControlNet conditioning
    control_inputs = {
        "control_image": masked_depth,  # The masked depth map as the ControlNet conditioning input
    }

    # Generate the inpainted image using Stable Diffusion with ControlNet conditioning
    inpainted_image = pipe(
        prompt=improved_caption,
        image=image,
        mask_image=mask,  # The damage mask to guide the inpainting
        controlnet=controlnet_model,
        controlnet_input=control_inputs,  # Depth conditioning input
        num_inference_steps=50,  # Number of inference steps
        guidance_scale=10,  # Controls the strength of prompt guidance
        strength=1  # How much to rely on the original image and mask
    ).images[0]

    # Create comparison plot
    fig, axes = plt.subplots(1, 6, figsize=(24, 5))

    # Original Image
    axes[0].imshow(image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    # Mask
    axes[1].imshow(mask, cmap="gray")
    axes[1].set_title("Generated Mask")
    axes[1].axis("off")

    # Masked Image
    axes[2].imshow(image)
    axes[2].imshow(mask, alpha=0.5, cmap="gray")
    axes[2].set_title("Mask Overlay")
    axes[2].axis("off")

    # Depth Map
    axes[3].imshow(depth_map, cmap="gray")
    axes[3].set_title("Depth Map")
    axes[3].axis("off")

    # Masked Depth Map
    axes[4].imshow(masked_depth, cmap="gray")
    axes[4].set_title("Masked Depth Map")
    axes[4].axis("off")

    # Inpainted Result
    axes[5].imshow(inpainted_image)
    axes[5].set_title("Inpainted Result")
    axes[5].axis("off")

    plt.show()



# Optional: Test random mask variations
test_single_sample(df, sample_index=random.randint(0, len(df)))

In [ ]:
# Specify where you want to save the pipeline
save_path = "/content/fine_tuned_pipeline"

# Save the pipeline to disk
pipe.save_pretrained(save_path)

# Optional: Save the model configuration
pipe.config.save_pretrained(save_path)


NameError: name 'pipe' is not defined

In [ ]:
# Example of saving the fine-tuned model
pipe.save_pretrained("path_to_save_model")

In [ ]:
from peft import LoraConfig

def prepare_lora(unet, rank=4):
    """Prepares a UNet model for LoRA training by adding PEFT LoRA adapters."""
    # Freeze all parameters first
    for param in unet.parameters():
        param.requires_grad = False

    # Create LoRA config
    lora_config = LoraConfig(
        r=rank,
        lora_alpha=rank,
        init_lora_weights="gaussian",
        target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    )

    # Move UNet to correct dtype/device if needed (optional)
    unet.to(dtype=torch.float32)

    # Add LoRA adapters
    unet.add_adapter(lora_config)

    # Make LoRA parameters trainable
    for param in unet.parameters():
        if param.requires_grad:
            param.data = param.data.float()  # ensure float32 if needed

    return unet

# Test the function
if __name__ == "__main__":
    import torch
    from diffusers import StableDiffusionInpaintPipeline

    print("Loading model...")
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16,
    )

    print("\nBefore LoRA preparation:")
    print(f"Total parameters: {sum(p.numel() for p in pipe.unet.parameters())}")
    print(f"Trainable parameters: {sum(p.numel() for p in pipe.unet.parameters() if p.requires_grad)}")

    print("\nPreparing LoRA...")
    pipe.unet = prepare_lora(pipe.unet, rank=4)

    print("\nAfter LoRA preparation:")
    print(f"Total parameters: {sum(p.numel() for p in pipe.unet.parameters())}")
    print(f"Trainable parameters: {sum(p.numel() for p in pipe.unet.parameters() if p.requires_grad)}")

    # Count LoRA parameters
    lora_layers = sum(1 for name, _ in pipe.unet.named_parameters() if "lora" in name.lower())
    print(f"\nNumber of LoRA layers applied: {lora_layers}")

    # Count attention processors
    print(f"Number of attention processors: {len(pipe.unet.attn_processors)}")


In [ ]:
import os
import torch
import logging
from datetime import datetime
from accelerate import Accelerator
from diffusers import DDPMScheduler, StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import math

In [ ]:
import logging
import os
import torch
import numpy as np
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from accelerate import Accelerator
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, DDPMScheduler
from torchvision import transforms
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def train_inpainting_lora(
    dataframe,
    output_dir="./output",
    pretrained_model_name="runwayml/stable-diffusion-inpainting",
    controlnet_model_name="lllyasviel/sd-controlnet-depth",
    resolution=512,
    train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-4,
    lr_warmup_steps=100,
    max_train_steps=1000,
    checkpointing_steps=500,
    validation_steps=250,
    rank=4,
    mixed_precision="fp16",
    use_xformers=True,
    seed=42,
):
    # Initialize accelerator
    accelerator = Accelerator(
        gradient_accumulation_steps=gradient_accumulation_steps,
        mixed_precision=mixed_precision,
        log_with="tensorboard",
        project_dir=os.path.join(output_dir, "logs"),
    )

    # Set seed and create directories
    torch.manual_seed(seed)
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "checkpoints"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "samples"), exist_ok=True)

    # Initialize models
    controlnet = ControlNetModel.from_pretrained(
        controlnet_model_name,
        torch_dtype=torch.float16
    ).to(accelerator.device)

    pipeline = StableDiffusionControlNetPipeline.from_pretrained(
        pretrained_model_name,
        controlnet=controlnet,
        torch_dtype=torch.float16
    ).to(accelerator.device)

    # Prepare LoRA
    def prepare_lora(unet, rank=4):
        # LoRA preparation logic here
        return unet
    pipeline.unet = prepare_lora(pipeline.unet, rank=rank).to(accelerator.device)

    # Freeze non-LoRA components
    pipeline.vae.requires_grad_(False)
    pipeline.text_encoder.requires_grad_(False)
    pipeline.controlnet.requires_grad_(False)

    # Enable xformers if available
    if use_xformers:
        pipeline.enable_xformers_memory_efficient_attention()

    # Dataset and DataLoader
    dataset = MonumentDataset(dataframe, pipeline.tokenizer, size=resolution)
    train_dataloader = DataLoader(
        dataset,
        batch_size=train_batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2
    )

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(pipeline.unet.parameters(), lr=learning_rate)
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=max_train_steps)
    noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name, subfolder="scheduler")

    # Prepare components with accelerator
    pipeline.unet, pipeline.controlnet, optimizer, train_dataloader, lr_scheduler, noise_scheduler = accelerator.prepare(
        pipeline.unet, pipeline.controlnet, optimizer, train_dataloader, lr_scheduler, noise_scheduler
    )

    # Initialize TensorBoard writer
    if accelerator.is_main_process:
        writer = SummaryWriter(log_dir=os.path.join(output_dir, "logs", datetime.now().strftime("%Y%m%d_%H%M%S")))

    # Training loop
    global_step = 0
    progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)

    while global_step < max_train_steps:
        pipeline.unet.train()

        for batch in train_dataloader:
            # In the training loop, modify the data loading and loss calculation:

            # In the training loop, modify the sequence of operations:

            # In the training loop, modify the data loading and loss calculation:

            with accelerator.accumulate(pipeline.unet):
                # Load data with proper dtype
                damaged_images = batch["damaged_images"].to(accelerator.device)
                original_images = batch["original_images"].to(accelerator.device)
                masks = batch["masks"].to(accelerator.device)
                masked_depth = batch["masked_depth"].to(accelerator.device)
                input_ids = batch["input_ids"].to(accelerator.device)

                # Convert images to latent space with autocast
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    # First encode images to latents
                    damaged_latents = pipeline.vae.encode(damaged_images).latent_dist.sample()
                    original_latents = pipeline.vae.encode(original_images).latent_dist.sample()
                    damaged_latents = damaged_latents * pipeline.vae.config.scaling_factor
                    original_latents = original_latents * pipeline.vae.config.scaling_factor

                    # Prepare mask for latent space
                    mask_latent = torch.nn.functional.interpolate(
                        masks,
                        size=(damaged_latents.shape[2], damaged_latents.shape[3]),
                        mode="nearest"
                    )

                    # Add noise to damaged latents
                    noise = torch.randn_like(damaged_latents)
                    timesteps = torch.randint(
                        0, noise_scheduler.config.num_train_timesteps,
                        (damaged_latents.shape[0],), device=damaged_latents.device
                    )
                    noisy_latents = noise_scheduler.add_noise(damaged_latents, noise, timesteps)

                    # Create 9-channel input for inpainting UNet
                    model_input = torch.cat([
                        noisy_latents,      # 4 channels (noisy version of damaged latents)
                        original_latents,   # 4 channels
                        mask_latent         # 1 channel
                    ], dim=1)

                    # Prepare ControlNet conditioning
                    controlnet_cond = torch.nn.functional.interpolate(
                        masked_depth,
                        size=(resolution, resolution),
                        mode="bilinear"
                    ).repeat(1, 3, 1, 1)

                    # Get text embeddings
                    encoder_hidden_states = pipeline.text_encoder(input_ids)[0]

                    # ControlNet forward
                    down_block_res_samples, mid_block_res_sample = pipeline.controlnet(
                        noisy_latents,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states,
                        controlnet_cond=controlnet_cond,
                        return_dict=False,
                    )

                    # UNet forward with proper input channels
                    noise_pred = pipeline.unet(
                        model_input,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states,
                        down_block_additional_residuals=down_block_res_samples,
                        mid_block_additional_residual=mid_block_res_sample,
                    ).sample

                    # Calculate loss with proper casting
                    mask_bool = mask_latent > 0.5
                    mask_expanded = mask_bool.repeat(1, noise_pred.shape[1], 1, 1)
                    loss = torch.nn.functional.mse_loss(
                        noise_pred[mask_expanded].float(),  # Convert to float32 for loss calculation
                        noise[mask_expanded].float()
                    )

                # Backward pass with proper scaling
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(pipeline.unet.parameters(), 1.0)

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Logging and validation
            if accelerator.is_main_process:
                if global_step % validation_steps == 0:
                    validate_and_log(pipeline, dataset, global_step, output_dir, writer)

                if global_step % checkpointing_steps == 0:
                    save_checkpoint(pipeline, output_dir, global_step, accelerator)

            progress_bar.update(1)
            global_step += 1

            if global_step >= max_train_steps:
                break

    # Final save
    if accelerator.is_main_process:
        save_checkpoint(pipeline, output_dir, global_step, accelerator, is_final=True)
        writer.close()
        logger.info("Training complete")

def collate_fn(examples):
    return {
        "damaged_images": torch.stack([ex["damaged_images"] for ex in examples]),
        "original_images": torch.stack([ex["original_images"] for ex in examples]),
        "masks": torch.stack([ex["masks"] for ex in examples]),
        "depth_map": torch.stack([ex["depth_map"] for ex in examples]),
        "masked_depth": torch.stack([ex["masked_depth"] for ex in examples]),
        "input_ids": torch.stack([ex["input_ids"] for ex in examples]),
        "metadata": [ex["metadata"] for ex in examples]
    }

class MonumentDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, size=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.size = size

        self.image_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

        self.mask_transform = transforms.Compose([
            transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST),
            transforms.CenterCrop(size),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        intact_image = Image.open(row['image_path']).convert("RGB")
        depth_map = Image.open(row['depth_path']).convert("L")

        # Generate random mask
        mask = random_damage_mask(intact_image.size, ratio=0.3)

        # Create damaged image
        damaged_image = Image.composite(
            Image.new("RGB", intact_image.size, (0,0,0)),
            intact_image,
            mask
        )

        return {
            "damaged_images": self.image_transform(damaged_image),
            "original_images": self.image_transform(intact_image),
            "masks": self.mask_transform(mask),
            "depth_map": self.mask_transform(depth_map),
            "masked_depth": self.mask_transform(depth_map) * (1 - self.mask_transform(mask)),
            "input_ids": self.tokenizer(
                improve_caption(row['raw_caption']),
                padding="max_length",
                max_length=self.tokenizer.model_max_length,
                truncation=True,
                return_tensors="pt"
            ).input_ids.squeeze(0),
            "metadata": {
                "raw_caption": row['raw_caption'],
                "original_image": intact_image,
                "generated_mask": mask,
                "depth_map": depth_map
            }
        }

def random_damage_mask(size, ratio=0.3):
    mask = Image.new("L", size, 0)
    draw = ImageDraw.Draw(mask)
    for _ in range(np.random.randint(3, 8)):
        x1 = np.random.randint(0, size[0])
        y1 = np.random.randint(0, size[1])
        x2 = x1 + np.random.randint(int(size[0]*0.1), int(size[0]*0.4))
        y2 = y1 + np.random.randint(int(size[1]*0.1), int(size[1]*0.4))
        draw.rectangle([x1, y1, x2, y2], fill=255)
    return mask

def improve_caption(raw_caption):
    return f"high quality photo of {raw_caption}, detailed texture, realistic lighting, 8k"

def validate_and_log(pipeline, dataset, step, output_dir, writer, num_samples=4):
    pipeline.eval()
    sample_dir = os.path.join(output_dir, "samples", f"step_{step}")
    os.makedirs(sample_dir, exist_ok=True)

    with torch.no_grad():
        for i in range(num_samples):
            example = dataset[np.random.randint(len(dataset))]
            metadata = example["metadata"]

            inpainted = pipeline(
                prompt=improve_caption(metadata["raw_caption"]),
                image=metadata["original_image"].resize((512, 512)),
                mask_image=metadata["generated_mask"],
                controlnet_conditioning_image=metadata["depth_map"].resize((512, 512)),
                num_inference_steps=50,
                guidance_scale=7.5,
                generator=torch.Generator(device=pipeline.device).manual_seed(42+i),
            ).images[0]

            fig, axs = plt.subplots(1, 5, figsize=(25, 5))
            axs[0].imshow(metadata["original_image"].resize((512, 512)))
            axs[1].imshow(metadata["generated_mask"], cmap="gray")
            axs[2].imshow(metadata["depth_map"], cmap="gray")
            axs[3].imshow(Image.composite(
                Image.new("L", metadata["depth_map"].size, 0),
                metadata["depth_map"],
                metadata["generated_mask"]
            ), cmap="gray")
            axs[4].imshow(inpainted)
            plt.savefig(os.path.join(sample_dir, f"sample_{i}.png"))
            plt.close()

def save_checkpoint(pipeline, output_dir, step, accelerator, is_final=False):
    save_path = os.path.join(output_dir, "checkpoints", f"checkpoint-{step}")
    os.makedirs(save_path, exist_ok=True)

    if is_final:
        pipeline.save_pretrained(save_path)
    else:
        lora_layers = {k: v for k, v in pipeline.unet.state_dict().items() if "lora" in k}
        torch.save({
            "global_step": step,
            "unet_lora": lora_layers,
            "optimizer": accelerator.get_state_dict(optimizer),
            "lr_scheduler": accelerator.get_state_dict(lr_scheduler),
        }, os.path.join(save_path, "checkpoint.pt"))

    logger.info(f"Saved checkpoint to {save_path}")

# Initialize and run training
train_inpainting_lora(dataframe=df)